1. 导入相关库

In [1]:
import re
import requests
from bs4 import BeautifulSoup

In [2]:
html_asdjkjfaskl = 'http://220.160.23.161:8090/sd_web/'
html_fdfdf = "http://220.160.23.161:8090/sd_web/lp_detailinfo.jsp?id=2019034&type=-1"

提取出福建五环开发商的温泉项目

虽然不能直接从这个网页的源代码提取每个预售证号对应的html，但是观察规律可以看到为对应的字符串加入预售证号，如下：

In [3]:
# function 1
# return list of 每个预售证号的html
html_1_list = []
for i in [2019036,2019034,2019033,2019032,2019029,2019027,2019024,2019012,2019006,2018036,2018034,2018032,2018030,2018015,2018013]:
    str_i = "http://220.160.23.161:8090/sd_web/lp_detailinfo.jsp?id={}&type=-1".format(str(i))
    # print(str_i)
    html_1_list.append(str_i)
# html_1_list为这个网页提取出来的每个预售证号的html
# http://220.160.23.161:8090/sd_web/

In [4]:
# function 2
# input: function 1 的输出，即每个预售证号的html
# return: list of 点击查看其它楼栋 的html

html = "http://220.160.23.161:8090/sd_web/lp_detailinfo.jsp?id=2019036&type=-1"
html = requests.get(html)
# print(html.text)
html = BeautifulSoup(html.text)

reg = re.compile(r'.*点击查看其它楼栋.*')
list_slse_build = html.find(text=reg).parent.find_all('a')
list_slse_build

html_2_list = []
for i in list_slse_build:
    insert_value = i['href'][-10:-1]
    str_build = "http://220.160.23.161:8090/sd_web/dy_list.jsp?l_bm={}".format(insert_value)
    html_2_list.append(str_build)
html_2_list

['http://220.160.23.161:8090/sd_web/dy_list.jsp?l_bm=100007736',
 'http://220.160.23.161:8090/sd_web/dy_list.jsp?l_bm=100007737']

上面的网址是点击查看其他楼栋的html，输出为一个列表，下面以第一个列表为例来访问每一个住宅

In [5]:
# function 3
# input: function 2 的输出，即其他楼栋的html列表
# output: 每一个其他楼栋中所有住房的html


html_3 = "http://220.160.23.161:8090/sd_web/dy_list.jsp?l_bm=100007736"
html_3 = requests.get(html_3)
html_3 = BeautifulSoup(html_3.text)
list_build = html_3.find_all(target="_blank")
# list_build
# list_build[0]['href'][-14:]
html_3_list = []
for i in list_build:
    # 这个insert_value其实就是单元编号
    insert_value = i['href'][-14:]
    str_build = "http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id={}".format(insert_value)
    html_3_list.append(str_build)
html_3_list
# http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0045


['http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0045',
 'http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0046',
 'http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0047',
 'http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0048',
 'http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0049',
 'http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0050',
 'http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0051',
 'http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0052',
 'http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0053',
 'http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0054',
 'http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0055',
 'http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0056',
 'http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0057',
 'http://220.160.23.161:8090/sd_web/dy

In [6]:
# function 4

html_4 = "http://220.160.23.161:8090/sd_web/dy_detailinfo.jsp?id=100007736-0045"
html_4 = requests.get(html_4)
html_4 = BeautifulSoup(html_4.text)

In [13]:
# 单元地址的信息是在head里面的，需要单独输出

unit_location = html_4.head.find_all('script')
unit_location = unit_location[2]
# unit_location = unit_location[2].get_text().strip()
# unit_location = unit_location.split()[1]
# unit_location = unit_location[4:-2]
unit_location

<script type="text/javascript">
		var ss="永泰县城峰镇汤洋村上山院6号世茂永泰温泉小镇云悦2幢01店面01店面";
		ss=ss.replace("佛山市顺德区","");
		$(document).ready(function(){
		  $("#sss1").html(ss);
		  });
		</script>

In [9]:
# 其余的信息是在 tbody 里面的，需要进一步处理
else_information = html_4.tbody
else_information = else_information.find_all('tr')
# else_information

In [10]:
else_information[0].get_text().split()

['单元编号:', '100007736-0045', '单元名称:', '01店面']

In [11]:
else_information_list = []
for i in else_information:
    else_information_i = i.get_text().split()
    else_information_list.append(else_information_i)
    print(i.get_text().split())

['单元编号:', '100007736-0045', '单元名称:', '01店面']
['单元地址:', '所在镇(村居):']
['房屋用途:', '商业服务', '房屋结构:', '钢混']
['单元建筑面积:', '31.74', '平方米', '公建分摊面积:', '.99', '平方米']
['套内建筑面积:', '30.75', '平方米', '单元状态:', '可售']
['可售']
[]
['参考价:', '39000', '(元/m²)']


In [12]:
unit_number = else_information_list[0][1] # 单元编号
unit_name = else_information_list[0][3] # 单元名称
house_use = else_information_list[2][0] # 房屋用途
unit_area = else_information_list[3][1] # 单元建筑面积
allocated_area_for_public_construction = else_information_list[3][4] # 公建分摊面积
the_floor_area_within_the_suite = else_information_list[4][1] # 套内建筑面积
unit_state = else_information_list[4][4] # 单元状态
price = else_information_list[7][1] # 参考价

In [16]:
print(' 单元编号: \t{}\n 单元名称: \t{}\n 房屋用途: \t{}\n 单元建筑面积: \t{}\n 公建分摊面积: \t{}\n 套内建筑面积: \t{}\n 单元状态: \t{}\n 参考价: \t{}'.format(
            unit_number,
            unit_name,
            house_use,
            unit_area,
            allocated_area_for_public_construction,
            the_floor_area_within_the_suite,
            unit_state,
            price
        ))

 单元编号: 	100007736-0045
 单元名称: 	01店面
 房屋用途: 	房屋用途:
 单元建筑面积: 	31.74
 公建分摊面积: 	.99
 套内建筑面积: 	30.75
 单元状态: 	可售
 参考价: 	39000
